In [54]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from imutils import paths
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl
IMG_X = 64
IMG_Y = 64
import joblib
from tensorflow.keras.models import load_model

In [26]:

def form_training_data(paths,cls):
    X = []
    y = []
    for imgpath,cl in zip(paths,cls):
        try:
            img_array = cv2.imread(imgpath,cv2.IMREAD_GRAYSCALE)
            img_array = cv2.resize(img_array,(IMG_X,IMG_Y))
            X.append(img_array)
            y.append(cl)
        except:
            pass
    
    return np.array(X).reshape(-1,IMG_X,IMG_Y,1),np.array(y)



In [20]:
imagePaths_test_0 = list(paths.list_images("./datasets/test_folder/0/"))
imagePaths_test_1 = list(paths.list_images("./datasets/test_folder/1/"))
imagePaths_test = imagePaths_test_0 + imagePaths_test_1
test_class = [0]*len(imagePaths_test_0)+[1]*len(imagePaths_test_1)
imagePaths_train_0 = list(paths.list_images("./datasets/train_folder/0/"))
imagePaths_train_1 = list(paths.list_images("./datasets/train_folder/1/"))
imagePaths_train = imagePaths_train_0 + imagePaths_train_1
train_class = [0]*len(imagePaths_train_0)+[1]*len(imagePaths_train_1)

X_train,y_train = form_training_data(imagePaths_train,train_class)
X_test,y_test = form_training_data(imagePaths_test,test_class)

In [22]:
#shuffle the training data
shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
y_train = y_train[shuffler]

In [44]:
model = tf.keras.Sequential([
    tfl.ZeroPadding2D(padding=3,input_shape = X_train.shape[1:],),
    tfl.Conv2D(64,7,strides=(1,1)),
    tfl.BatchNormalization(axis = -1),
    tfl.ReLU(),
    tfl.MaxPool2D(),
    tfl.Flatten(),
    tfl.Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_3 (ZeroPaddi  (None, 70, 70, 1)        0         
 ng2D)                                                           
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 64)        3200      
                                                                 
 batch_normalization_3 (Batc  (None, 64, 64, 64)       256       
 hNormalization)                                                 
                                                                 
 re_lu_3 (ReLU)              (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                      

In [45]:

model.compile(optimizer="adam",loss="binary_crossentropy",metrics = ['accuracy'])

In [60]:
model.fit(X_train,y_train,epochs=100,batch_size=32,validation_split=0.1)

Epoch 1/100
79/79 [==============================] - 15s 188ms/step - loss: 0.1682 - accuracy: 0.9345 - val_loss: 0.8207 - val_accuracy: 0.7857
Epoch 2/100
79/79 [==============================] - 16s 207ms/step - loss: 0.1884 - accuracy: 0.9250 - val_loss: 0.8139 - val_accuracy: 0.7857
Epoch 3/100
79/79 [==============================] - 15s 184ms/step - loss: 0.1641 - accuracy: 0.9353 - val_loss: 0.5058 - val_accuracy: 0.8393
Epoch 4/100
79/79 [==============================] - 14s 177ms/step - loss: 0.1250 - accuracy: 0.9488 - val_loss: 0.6320 - val_accuracy: 0.8214
Epoch 5/100
79/79 [==============================] - 14s 177ms/step - loss: 0.0824 - accuracy: 0.9730 - val_loss: 0.6191 - val_accuracy: 0.8250
Epoch 6/100
79/79 [==============================] - 14s 177ms/step - loss: 0.0806 - accuracy: 0.9710 - val_loss: 1.1420 - val_accuracy: 0.7464
Epoch 7/100
79/79 [==============================] - 14s 176ms/step - loss: 0.0659 - accuracy: 0.9770 - val_loss: 0.6217 - val_accuracy:

In [61]:
model.save("./Model")

INFO:tensorflow:Assets written to: ./Model\assets


In [62]:
model_ = load_model("./Model")

In [63]:
model_.evaluate(X_test,y_test)

38/38 [==============================] - 2s 50ms/step - loss: 0.8994 - accuracy: 0.8450


[0.8994207978248596, 0.8450000286102295]